In [ ]:
pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.0/224.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.7/285.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.3/372.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 8.5 MB/s 

In [ ]:
import pandas as pd
import numpy as np
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

In [ ]:
from autogluon.core import Categorical

In [ ]:
from autogluon.tabular import TabularPredictor

In [ ]:
merge=pd.read_csv('merge.csv')

In [ ]:
merge1 = merge['trade']

In [ ]:
merge2= pd.concat([train_df, merge1], axis=1)

In [ ]:
merge2

,ID,timestamp,supply(kg),price(원/kg),item_id,trade
0,TG_A_J_20190101,2019-01-01,0.0,0.0,TG_A_J,172.0
1,TG_A_J_20190102,2019-01-02,0.0,0.0,TG_A_J,172.0
2,TG_A_J_20190103,2019-01-03,60601.0,1728.0,TG_A_J,172.0
3,TG_A_J_20190104,2019-01-04,25000.0,1408.0,TG_A_J,172.0
4,TG_A_J_20190105,2019-01-05,32352.0,1250.0,TG_A_J,172.0
...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,452440.0,468.0,RD_F_J,0.0
59393,RD_F_J_20230228,2023-02-28,421980.0,531.0,RD_F_J,0.0
59394,RD_F_J_20230301,2023-03-01,382980.0,574.0,RD_F_J,0.0
59395,RD_F_J_20230302,2023-03-02,477220.0,523.0,RD_F_J,0.0


In [ ]:
train_df = pd.read_csv('train.csv')[['ID','timestamp','supply(kg)', 'price(원/kg)']]
#test_df = pd.read_csv('test.csv')[['ID','timestamp']]
train_df['item_id'] = train_df.ID.str[0:6]
#test_df['item_id'] = test_df.ID.str[0:6]

In [ ]:
train_df = merge2[['ID','timestamp','supply(kg)', 'price(원/kg)','trade']]
# test_df = pd.read_csv('test.csv')[['ID','timestamp']]
train_df['item_id'] = train_df.ID.str[0:6]
# test_df['item_id'] = test_df.ID.str[0:6]

In [ ]:
data = TimeSeriesDataFrame(train_df.drop(columns=['ID']))
predictor = TimeSeriesPredictor(
    prediction_length=28,
    target="price(원/kg)",
    eval_metric="RMSE",

)
hyperparameters = {
    "DeepAR": {},  # DeepAR 모델의 하이퍼파라미터 설정
    "ETS": {"seasonal": "add"},  # ETS 모델의 하이퍼파라미터 설정
    "AutoARIMA":{},
    "SeasonalNaive":{},
    "TemporalFusionTransformer":{},
    "DirectTabular":{},
    "AutoETS":{},
    "RecursiveTabular":{},
    "Theta":{},}
# seed 고정
predictor.fit( data, random_seed=42, hyperparameters=hyperparameters,presets="high_quality")

================ TimeSeriesPredictor ================
TimeSeriesPredictor.fit() called
Setting presets to: high_quality
Fitting with arguments:
{'enable_ensemble': True,
 'evaluation_metric': 'RMSE',
 'excluded_model_types': None,
 'hyperparameter_tune_kwargs': None,
 'hyperparameters': {'AutoARIMA': {},
                     'AutoETS': {},
                     'DeepAR': {},
                     'DirectTabular': {},
                     'ETS': {'seasonal': 'add'},
                     'RecursiveTabular': {},
                     'SeasonalNaive': {},
                     'TemporalFusionTransformer': {},
                     'Theta': {}},
 'num_val_windows': 1,
 'prediction_length': 28,
 'random_seed': 42,
 'target': 'price(원/kg)',
 'time_limit': None,
 'verbosity': 2}
Provided training data set with 59397 rows, 39 items (item = single time series). Average time series length is 1523.0. Data frequency is 'D'.
INFO:lightning_fabric.utilities.seed:Global seed set to 42
AutoGluon will save m

In [ ]:
# seed 고정
pred = predictor.predict(data, random_seed=42, )

INFO:lightning_fabric.utilities.seed:Global seed set to 42
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble


In [ ]:
pred

mean          0.1          0.2          0.3  \
item_id timestamp                                                        
TG_A_J  2023-03-04  3522.901876 -1813.470938   148.086566  1414.040426   
        2023-03-05   554.548245 -4474.032904 -2746.041455 -1513.446190   
        2023-03-06  3379.931528 -2228.988995  -193.529078  1130.378950   
        2023-03-07  3079.259048 -2706.510933  -588.033825   775.382187   
        2023-03-08  3358.184087 -2495.445781  -380.235893  1004.853380   
...                         ...          ...          ...          ...   
RD_F_J  2023-03-27   571.288644  -145.200432   123.035664   291.951445   
        2023-03-28   572.603558  -147.493561   117.473420   287.375131   
        2023-03-29   566.523117  -151.878957   112.446159   282.306708   
        2023-03-30   562.594036  -167.719026   101.154327   273.332529   
        2023-03-31   581.010369  -163.145953   113.835382   287.403241   

                            0.4          0.5          0.6          0.7  \
item_id timestamp                                                        
TG_A_J  2023-03-04  2501.041298  3517.615132  4530.640444  5633.104267   
        2023-03-05  -442.196122   552.893440  1545.767203  2610.428599   
        2023-03-06  2304.225953  3397.424631  4488.927823  5675.389724   
        2023-03-07  1961.839797  3077.220108  4191.429337  5402.376703   
        2023-03-08  2231.671428  3366.143071  4490.076201  5732.028160   
...                         ...          ...          ...          ...   
RD_F_J  2023-03-27   441.912592   573.823190   715.941815   866.782563   
        2023-03-28   439.782474   572.447118   717.295294   869.441902   
        2023-03-29   435.852800   569.915795   714.053855   865.961931   
        2023-03-30   428.742622   562.752514   710.375970   864.247634   
        2023-03-31   445.132586   580.799126   728.487750   880.093384   

                            0.8          0.9  
item_id timestamp                             
TG_A_J  2023-03-04  6879.644451  8672.994660  
        2023-03-05  3854.112158  5587.659539  
        2023-03-06  6981.073978  8883.254048  
        2023-03-07  6752.224265  8707.911893  
        2023-03-08  7119.202192  9112.201681  
...                         ...          ...  
RD_F_J  2023-03-27  1033.520747  1280.156288  
        2023-03-28  1038.382485  1285.366110  
        2023-03-29  1035.951668  1286.275774  
        2023-03-30  1037.549310  1289.320715  
        2023-03-31  1056.731403  1310.564255  

[1092 rows x 10 columns]

In [ ]:
ld_board = predictor.leaderboard(train_df, silent=True)

Additional data provided, testing on additional data. Resulting leaderboard will be sorted according to test score (`score_test`).


In [ ]:
ld_board

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time_marginal,fit_order
0,WeightedEnsemble,-670.664288,-670.518469,43.744744,60.758589,8.632991,11
1,TemporalFusionTransformer,-716.578875,-716.578875,0.230040,0.230877,1294.921071,7
2,DeepAR,-764.572387,-764.197427,2.419911,5.085151,679.437069,6
3,DirectTabular,-793.949226,-793.949226,0.188235,0.108351,6.485334,9
4,AutoETS,-797.059173,-797.059173,40.581418,55.053452,0.077040,4
5,RecursiveTabular,-810.212523,-810.212523,0.852150,1.748199,16.095783,5
6,Theta,-838.834688,-838.834688,45.658539,94.056434,0.072065,3
7,SeasonalNaive,-843.925825,-843.925825,0.117929,0.124551,0.069782,2
8,AutoARIMA,-845.688839,-845.688839,310.893467,320.936541,0.073753,10
9,Naive,-997.063356,-997.063356,0.190500,0.156206,0.095282,1


In [ ]:
submission = pd.read_csv('./sample_submission.csv')
submission['answer'] = pred.reset_index()['mean']
submission.loc[ submission['answer'] < 0.0, 'answer'] = 0.0
submission.to_csv('./auto2_submission.csv', index=False)
submission

,ID,answer
0,TG_A_J_20230304,3296.538252
1,TG_A_J_20230305,603.187818
2,TG_A_J_20230306,3091.974528
3,TG_A_J_20230307,3529.243130
4,TG_A_J_20230308,3408.026579
...,...,...
1087,RD_F_J_20230327,513.321159
1088,RD_F_J_20230328,541.252372
1089,RD_F_J_20230329,544.994414
1090,RD_F_J_20230330,523.648052
